# 01 - Exploratory Data Analysis (EDA)
## Real Estate Price Prediction

**Author:** Nicolas  
**Date:** 2025-01-09  
**Objective:** Analyser les données immobilières et identifier les patterns pour la prédiction des prix

---

### Table of Contents
1. [Data Loading](#1-data-loading)
2. [Data Overview](#2-data-overview)
3. [Missing Values Analysis](#3-missing-values-analysis)
4. [Univariate Analysis](#4-univariate-analysis)
5. [Bivariate Analysis](#5-bivariate-analysis)
6. [Correlation Analysis](#6-correlation-analysis)
7. [Outlier Detection](#7-outlier-detection)
8. [Geographical Analysis](#8-geographical-analysis)
9. [Key Insights & Next Steps](#9-key-insights)

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import warnings

warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

%matplotlib inline

## 1. Data Loading

### Data Source Options:
Pour ce projet, vous pouvez utiliser plusieurs sources:

1. **DVF Open Data (France)** - Recommandé
   - Source: https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/
   - Données réelles de transactions immobilières en France
   - Format: CSV, millions de transactions

2. **Kaggle Datasets**
   - House Prices: Advanced Regression Techniques
   - California Housing Dataset

3. **Dataset synthétique** (pour démonstration rapide)
   - Généré ci-dessous pour tester le pipeline

In [ ]:
# Fonction pour générer un dataset synthétique (à remplacer par vraies données)
def generate_synthetic_data(n_samples=5000, random_state=42):
    """
    Génère des données synthétiques pour tester le pipeline.
    À REMPLACER par des vraies données DVF ou Kaggle.
    """
    np.random.seed(random_state)
    
    # Villes françaises
    cities = ['Paris', 'Lyon', 'Marseille', 'Bordeaux', 'Lille', 'Nice', 'Toulouse', 'Nantes']
    city_base_prices = [10000, 4500, 3800, 4200, 3500, 5000, 4000, 4300]
    
    data = []
    for _ in range(n_samples):
        city_idx = np.random.choice(len(cities))
        city = cities[city_idx]
        base_price = city_base_prices[city_idx]
        
        # Features
        surface = np.random.gamma(shape=5, scale=15) + 20  # 20-150 m²
        rooms = int(np.clip(surface / 30 + np.random.normal(0, 0.5), 1, 7))
        age = np.random.exponential(scale=20)  # Age du bien
        floor = np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8], p=[0.15, 0.15, 0.15, 0.15, 0.1, 0.1, 0.1, 0.05, 0.05])
        has_elevator = 1 if floor > 2 and np.random.random() > 0.3 else 0
        has_parking = np.random.choice([0, 1], p=[0.6, 0.4])
        has_balcony = np.random.choice([0, 1], p=[0.5, 0.5])
        energy_class = np.random.choice(['A', 'B', 'C', 'D', 'E', 'F', 'G'], 
                                       p=[0.05, 0.10, 0.20, 0.30, 0.20, 0.10, 0.05])
        
        # Prix avec logique métier
        price = base_price * surface
        price *= (1 - age * 0.003)  # Dépréciation
        price *= (1 + has_elevator * 0.05)
        price *= (1 + has_parking * 0.08)
        price *= (1 + has_balcony * 0.03)
        
        energy_multiplier = {'A': 1.06, 'B': 1.03, 'C': 1.0, 'D': 0.97, 'E': 0.92, 'F': 0.88, 'G': 0.82}
        price *= energy_multiplier[energy_class]
        
        # Ajout de bruit
        price *= np.random.normal(1, 0.1)
        price = max(50000, price)  # Prix minimum
        
        data.append({
            'city': city,
            'surface_m2': round(surface, 1),
            'rooms': rooms,
            'age_years': round(age, 1),
            'floor': floor,
            'has_elevator': has_elevator,
            'has_parking': has_parking,
            'has_balcony': has_balcony,
            'energy_class': energy_class,
            'price': round(price, 2)
        })
    
    return pd.DataFrame(data)

# Charger ou générer les données
# Option 1: Charger vraies données
# df = pd.read_csv('../data/dvf_data.csv')

# Option 2: Dataset synthétique (pour test)
df = generate_synthetic_data(n_samples=5000)

# Sauvegarder
df.to_csv('../data/real_estate_data.csv', index=False)

print(f"Dataset chargé: {df.shape[0]} lignes, {df.shape[1]} colonnes")

## 2. Data Overview

In [ ]:
# Aperçu des données
print("=" * 80)
print("APERÇU DES DONNÉES")
print("=" * 80)
df.head(10)

In [ ]:
# Informations sur le dataset
print("\n" + "=" * 80)
print("INFORMATIONS SUR LE DATASET")
print("=" * 80)
df.info()

In [ ]:
# Statistiques descriptives
print("\n" + "=" * 80)
print("STATISTIQUES DESCRIPTIVES")
print("=" * 80)
df.describe()

## 3. Missing Values Analysis

In [ ]:
# Analyse des valeurs manquantes
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
}).sort_values('Percentage', ascending=False)

print("\n" + "=" * 80)
print("VALEURS MANQUANTES")
print("=" * 80)
print(missing_df[missing_df['Missing Count'] > 0])

# Visualisation
if missing_df['Missing Count'].sum() > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    missing_df[missing_df['Missing Count'] > 0].plot(kind='bar', y='Percentage', ax=ax)
    ax.set_title('Pourcentage de valeurs manquantes par variable', fontsize=14, fontweight='bold')
    ax.set_ylabel('Pourcentage (%)')
    ax.set_xlabel('Variables')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("✅ Aucune valeur manquante détectée!")

## 4. Univariate Analysis

In [ ]:
# Distribution de la variable cible: PRICE
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Histogramme
axes[0].hist(df['price'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution des Prix', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Prix (€)')
axes[0].set_ylabel('Fréquence')
axes[0].axvline(df['price'].mean(), color='red', linestyle='--', label=f'Moyenne: {df["price"].mean():,.0f}€')
axes[0].axvline(df['price'].median(), color='green', linestyle='--', label=f'Médiane: {df["price"].median():,.0f}€')
axes[0].legend()

# Boxplot
axes[1].boxplot(df['price'], vert=True)
axes[1].set_title('Boxplot des Prix', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Prix (€)')

# Q-Q plot
stats.probplot(df['price'], dist="norm", plot=axes[2])
axes[2].set_title('Q-Q Plot (Test de Normalité)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# Statistiques
print("\n" + "=" * 80)
print("STATISTIQUES - PRIX")
print("=" * 80)
print(f"Moyenne:     {df['price'].mean():,.2f} €")
print(f"Médiane:     {df['price'].median():,.2f} €")
print(f"Écart-type:  {df['price'].std():,.2f} €")
print(f"Min:         {df['price'].min():,.2f} €")
print(f"Max:         {df['price'].max():,.2f} €")
print(f"Skewness:    {df['price'].skew():.3f}")
print(f"Kurtosis:    {df['price'].kurtosis():.3f}")

In [ ]:
# Distribution des variables numériques
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove('price')  # Déjà analysé

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, col in enumerate(numeric_cols[:6]):
    axes[idx].hist(df[col], bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribution: {col}', fontsize=11, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Fréquence')
    axes[idx].axvline(df[col].mean(), color='red', linestyle='--', linewidth=1.5, label='Moyenne')
    axes[idx].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Distribution des variables catégorielles
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

for col in categorical_cols:
    fig, ax = plt.subplots(figsize=(10, 5))
    value_counts = df[col].value_counts()
    value_counts.plot(kind='bar', ax=ax, edgecolor='black', alpha=0.7)
    ax.set_title(f'Distribution: {col}', fontsize=12, fontweight='bold')
    ax.set_xlabel(col)
    ax.set_ylabel('Fréquence')
    plt.xticks(rotation=45, ha='right')
    
    # Ajouter les pourcentages
    for i, v in enumerate(value_counts):
        ax.text(i, v + max(value_counts)*0.01, f'{v}\n({v/len(df)*100:.1f}%)', 
                ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()

## 5. Bivariate Analysis

In [ ]:
# Prix vs Surface (relation la plus importante)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot
axes[0].scatter(df['surface_m2'], df['price'], alpha=0.5, s=20)
axes[0].set_title('Prix vs Surface', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Surface (m²)')
axes[0].set_ylabel('Prix (€)')

# Avec regression line
z = np.polyfit(df['surface_m2'], df['price'], 1)
p = np.poly1d(z)
axes[0].plot(df['surface_m2'], p(df['surface_m2']), "r--", linewidth=2, label='Régression linéaire')
axes[0].legend()

# Prix par m²
df['price_per_m2'] = df['price'] / df['surface_m2']
axes[1].hist(df['price_per_m2'], bins=50, edgecolor='black', alpha=0.7)
axes[1].set_title('Distribution du Prix au m²', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Prix/m² (€)')
axes[1].set_ylabel('Fréquence')
axes[1].axvline(df['price_per_m2'].mean(), color='red', linestyle='--', 
                label=f'Moyenne: {df["price_per_m2"].mean():,.0f}€/m²')
axes[1].legend()

plt.tight_layout()
plt.show()

# Corrélation
corr = df[['surface_m2', 'price']].corr().iloc[0, 1]
print(f"\nCorrélation Surface-Prix: {corr:.3f}")

In [ ]:
# Prix par ville
fig = px.box(df, x='city', y='price', color='city',
             title='Distribution des Prix par Ville',
             labels={'price': 'Prix (€)', 'city': 'Ville'})
fig.update_layout(showlegend=False, height=500)
fig.show()

# Statistiques par ville
print("\n" + "=" * 80)
print("PRIX MOYEN PAR VILLE")
print("=" * 80)
city_stats = df.groupby('city')['price'].agg(['mean', 'median', 'std', 'count']).round(0)
city_stats = city_stats.sort_values('mean', ascending=False)
print(city_stats)

In [ ]:
# Prix vs Classe énergétique
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Boxplot
df.boxplot(column='price', by='energy_class', ax=axes[0])
axes[0].set_title('Prix par Classe Énergétique', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Classe Énergétique')
axes[0].set_ylabel('Prix (€)')
plt.sca(axes[0])
plt.xticks(rotation=0)

# Prix moyen par classe
energy_price = df.groupby('energy_class')['price'].mean().sort_index()
axes[1].bar(energy_price.index, energy_price.values, edgecolor='black', alpha=0.7)
axes[1].set_title('Prix Moyen par Classe Énergétique', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Classe Énergétique')
axes[1].set_ylabel('Prix Moyen (€)')

for i, v in enumerate(energy_price.values):
    axes[1].text(i, v + max(energy_price)*0.01, f'{v:,.0f}€', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Impact des features binaires
binary_features = ['has_elevator', 'has_parking', 'has_balcony']
feature_names = ['Ascenseur', 'Parking', 'Balcon']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (feat, name) in enumerate(zip(binary_features, feature_names)):
    price_comparison = df.groupby(feat)['price'].mean()
    axes[idx].bar(['Sans', 'Avec'], price_comparison.values, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Impact: {name}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Prix Moyen (€)')
    
    # Calculer la différence en %
    diff_pct = ((price_comparison[1] - price_comparison[0]) / price_comparison[0]) * 100
    axes[idx].text(0.5, max(price_comparison)*0.95, 
                   f'+{diff_pct:.1f}%', ha='center', fontsize=14, 
                   bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
    
    for i, v in enumerate(price_comparison.values):
        axes[idx].text(i, v + max(price_comparison)*0.02, f'{v:,.0f}€', 
                       ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

## 6. Correlation Analysis

In [ ]:
# Matrice de corrélation
numeric_df = df.select_dtypes(include=[np.number])
correlation_matrix = numeric_df.corr()

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Matrice de Corrélation', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Top corrélations avec le prix
print("\n" + "=" * 80)
print("CORRÉLATIONS AVEC LE PRIX")
print("=" * 80)
price_corr = correlation_matrix['price'].sort_values(ascending=False)
print(price_corr)

## 7. Outlier Detection

In [ ]:
# Détection d'outliers avec IQR method
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Analyse pour le prix
outliers_price, lower, upper = detect_outliers_iqr(df, 'price')

print("=" * 80)
print("DÉTECTION D'OUTLIERS - PRIX")
print("=" * 80)
print(f"Borne inférieure: {lower:,.2f} €")
print(f"Borne supérieure: {upper:,.2f} €")
print(f"Nombre d'outliers: {len(outliers_price)} ({len(outliers_price)/len(df)*100:.2f}%)")

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Boxplot avec outliers
axes[0].boxplot(df['price'], vert=True)
axes[0].set_title('Boxplot des Prix (avec outliers)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Prix (€)')

# Distribution sans outliers
df_no_outliers = df[(df['price'] >= lower) & (df['price'] <= upper)]
axes[1].hist(df_no_outliers['price'], bins=50, edgecolor='black', alpha=0.7)
axes[1].set_title('Distribution des Prix (sans outliers)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Prix (€)')
axes[1].set_ylabel('Fréquence')

plt.tight_layout()
plt.show()

## 8. Geographical Analysis

In [ ]:
# Prix moyen par ville avec volume de transactions
city_analysis = df.groupby('city').agg({
    'price': ['mean', 'median', 'std'],
    'price_per_m2': 'mean',
    'city': 'count'
}).round(0)

city_analysis.columns = ['Prix Moyen', 'Prix Médian', 'Écart-type', 'Prix/m² Moyen', 'Nb Transactions']
city_analysis = city_analysis.sort_values('Prix Moyen', ascending=False)

print("\n" + "=" * 80)
print("ANALYSE GÉOGRAPHIQUE")
print("=" * 80)
print(city_analysis)

# Visualisation
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Prix moyen par ville
city_analysis['Prix Moyen'].plot(kind='barh', ax=axes[0], edgecolor='black', alpha=0.7)
axes[0].set_title('Prix Moyen par Ville', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Prix Moyen (€)')
axes[0].set_ylabel('Ville')

# Prix au m² par ville
city_analysis['Prix/m² Moyen'].plot(kind='barh', ax=axes[1], edgecolor='black', alpha=0.7, color='coral')
axes[1].set_title('Prix Moyen au m² par Ville', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Prix/m² Moyen (€)')
axes[1].set_ylabel('Ville')

plt.tight_layout()
plt.show()

## 9. Key Insights & Next Steps

In [ ]:
print("="*80)
print("KEY INSIGHTS")
print("="*80)

insights = """
1. DISTRIBUTION DU PRIX:
   - La distribution est asymétrique (skewed right)
   - Présence d'outliers significatifs
   - Transformation log pourrait être nécessaire

2. FEATURES IMPORTANTES:
   - Surface: corrélation la plus forte avec le prix
   - Ville: impact majeur (Paris >> autres villes)
   - Classe énergétique: différence notable (A vs G)
   - Parking/Ascenseur: impact moyen (+5-8%)

3. QUALITÉ DES DONNÉES:
   - Pas de valeurs manquantes (dataset synthétique)
   - Variables cohérentes
   - Outliers à traiter

4. RELATIONS NON-LINÉAIRES:
   - Âge vs Prix: dépréciation non-linéaire
   - Surface vs Prix: potentiellement non-linéaire

NEXT STEPS (Feature Engineering):
□ Créer des features d'interaction (surface × ville)
□ Encoder les variables catégorielles
□ Normaliser/Standardiser les features numériques
□ Créer des bins pour l'âge et la surface
□ Gérer les outliers (cap ou suppression)
□ Tester transformations (log, sqrt) sur le prix
"""

print(insights)

# Sauvegarder un rapport
report = {
    'dataset_shape': df.shape,
    'missing_values': df.isnull().sum().sum(),
    'price_mean': df['price'].mean(),
    'price_std': df['price'].std(),
    'outliers_count': len(outliers_price),
    'top_correlations': price_corr.to_dict()
}

import json
with open('../models/eda_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print("\n✅ Rapport EDA sauvegardé: ../models/eda_report.json")

---
## Conclusion

Ce notebook a permis d'explorer en profondeur le dataset immobilier. Les principales conclusions sont:

1. **Surface** est le prédicteur le plus important
2. **Ville** a un impact majeur (effet géographique)
3. **Classe énergétique** influence significativement le prix
4. Des **outliers** sont présents et nécessitent un traitement
5. Le prix pourrait bénéficier d'une **transformation logarithmique**

**Prochaine étape:** Notebook 02 - Feature Engineering

---